# 6.12.3. Función de tensión de Love en coordenadas axisimétricas

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez Marín, <daalvarez@unal.edu.co>  | Marzo 25, 2023      | Adaptando el código de MAXIMA a PYTHON |

In [1]:
from sympy import init_printing, latex, symbols, diff, Function, solve, expand, simplify, factor

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Se definen las variables y funciones simbólicas necesarias:

In [2]:
r, z, E, nu = symbols('r, z, E, nu')
phi         = Function('phi')(r, z)    # función de tensión de Love

Se define el laplaciano en coordenadas axisimétricas:
\begin{equation}
\nabla^2\phi = \frac{1}{r}\frac{\partial}{\partial r}\left(r\frac{\partial\phi}{\partial r}\right) + \frac{\partial^2\phi}{\partial z^2} = \frac{\partial^2 \phi}{\partial r^2} + \frac{1}{r}\frac{\partial \phi}{\partial r} + \frac{\partial^2\phi}{\partial z^2}
\end{equation}

In [3]:
lapl = lambda f : (1/r)*diff(r*diff(f,r),r) + diff(f,z,2)

Se calcula el biarmónico $\nabla^4\phi$ de la función de tensión de Love:

In [4]:
bphi = expand(lapl(lapl(phi)))

Se definen los esfuerzos utilizando la función de tensión de Love:
\begin{align}    
    \sigma_r(r,z)      &= \frac{\partial}{\partial z}\left(\nu\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial r^2}\right)\\
    \sigma_\theta(r,z) &= \frac{\partial}{\partial z}\left(\nu\nabla^2\phi(r,z) - \frac{1}{r}\frac{\partial \phi(r,z)}{\partial r}\right)\\
    \sigma_z(r,z)      &= \frac{\partial}{\partial z}\left((2-\nu)\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial z^2}\right)\\
    \tau_{rz}(r,z)     &= \frac{\partial}{\partial r}\left((1-\nu)\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial z^2}\right)
\end{align}

In [5]:
sr  = diff(nu*lapl(phi) - diff(phi,r,2),     z)
st  = diff(nu*lapl(phi) - (1/r)*diff(phi,r), z)
sz  = diff((2-nu)*lapl(phi) - diff(phi,z,2), z)
trz = diff((1-nu)*lapl(phi) - diff(phi,z,2), r)

Se definen los desplazamientos utilizando la función de tensión de Love:
\begin{align}
u_r(r,z) &= -\frac{1+\nu}{E}\frac{\partial^2 \phi(r,z)}{\partial r \partial z}\\
w(r,z) &= \frac{1+\nu}{E}\left[2(1-\nu)\nabla^2\phi(r,z) - \frac{\partial^2 \phi(r,z)}{\partial z^2}\right].
\end{align}

In [6]:
ur = -((1+nu)/E)*diff(phi, r,1, z,1)
w  =  ((1+nu)/E)*(2*(1-nu)*lapl(phi) - diff(phi,z,2))

Se reemplazan los esfuerzos en las ecuaciones de equilibrio, para el caso axisimétrico, haciendo la fuerza másica constante ($b_r = b_\theta = 0$):
\begin{align}
    \frac{\partial \sigma_r}{\partial r} +  \frac{\partial \tau_{rz}}{\partial z} + \frac{\sigma_r - \sigma_\theta}{r} + b_r &= 0 \\
    \frac{\partial \tau_{rz}}{\partial r} + \frac{\partial \sigma_z}{\partial z} + \frac{\tau_{rz}}{r} + b_z &= 0
\end{align}
para verificar que estas ecuaciones se convierten respectivamente en:
\begin{align}
    0 &= 0 \\
    (1 - \nu)\nabla^4\phi(r,z) &= 0
\end{align}

In [7]:
# Primera ecuación de equilibrio:
eq1 = expand(diff(sr,r) + diff(trz,z) + (sr - st)/r) # == 0
imprimir("", eq1, " = 0")

<IPython.core.display.Math object>

In [8]:
# Segunda ecuación de equilibrio:
eq2 = diff(trz,r) + diff(sz,z) + trz/r # == 0
imprimir("", expand(eq2 - (1-nu)*bphi), " = 0")

<IPython.core.display.Math object>

Se reemplazan los esfuerzos en las ecuaciones de compatibilidad bidimensionales en términos de esfuerzos, asumiendo que las fuerzas másicas son constantes, para el caso axisimétrico:
\begin{align}
\nabla^2 \sigma_r - \frac{2}{r^2}(\sigma_r - \sigma_\theta) + \frac{1}{1+\nu}\frac{\partial^2 \Theta}{\partial r^2} &= 0\\
\nabla^2 \sigma_\theta + \frac{2}{r^2}(\sigma_r - \sigma_\theta) + \frac{1}{1+\nu}\frac{1}{r}\frac{\partial \Theta}{\partial r} &= 0\\
\nabla^2 \sigma_z + \frac{1}{1+\nu}\frac{\partial ^2\Theta}{\partial z^2} &= 0\\
\nabla^2 \tau_{rz} - \frac{1}{r^2}\tau_{rz} + \frac{1}{1+\nu}\frac{\partial^2 \Theta}{\partial r \partial z} &= 0
\end{align}

In [9]:
Theta = sr + st + sz
co1 = lapl(sr) - (2/r**2)*(sr - st) + (1/(1+nu))*diff(Theta,r,2)
co2 = lapl(st) + (2/r**2)*(sr - st) + (1/(1+nu))*(1/r)*diff(Theta,r)
co3 = lapl(sz) + (1/(1+nu))*diff(Theta,z,2)
co4 = lapl(trz) - (1/r**2)*trz + (1/(1+nu))*diff(Theta,r,1,z,1)

Se verifica que las ecuaciones de compatibilidad anteriores se convierten en:
\begin{align}
    \nu \frac{\partial}{\partial z}\left(\nabla^4 \phi(r,z) \right)     &= 0\\
    \nu \frac{\partial}{\partial z}\left(\nabla^4 \phi(r,z) \right)     &= 0\\
    (2-\nu) \frac{\partial}{\partial z}\left(\nabla^4 \phi(r,z) \right) &= 0\\
    (1-\nu) \frac{\partial}{\partial r}\left(\nabla^4 \phi(r,z) \right) &= 0
\end{align}

In [10]:
imprimir("", simplify(co1 - nu*diff(bphi,z)), " = 0")

<IPython.core.display.Math object>

In [11]:
imprimir("", simplify(co2 - nu*diff(bphi,z)), " = 0")

<IPython.core.display.Math object>

In [12]:
imprimir("", simplify(co3 - (2-nu)*diff(bphi,z)), " = 0")

<IPython.core.display.Math object>

In [13]:
imprimir("", simplify(co4 - (1-nu)*diff(bphi,r)), " = 0")

<IPython.core.display.Math object>

Se escriben las ecuaciones de Cauchy-Navier para el caso axisimétrico
\begin{align}
    (\lambda+G) \frac{\partial e}{\partial r} + G\left(\nabla^2 u_r - \frac{u_r}{r^2} \right) + b_r &= 0 \\
    (\lambda+G) \frac{\partial e}{\partial z} + G \nabla^2 w  + b_z &= 0
\end{align}
en términos de la función de tensión de Love:

In [14]:
# Las constantes de Lamé:
lamda = E*nu/((1 + nu)*(1 - 2*nu))
G     = E/(2*(1 + nu))

# Dilatación cúbica para el caso axisimétrico:
e = diff(ur,r) + ur/r + diff(w,z)

# Se reemplazan los desplazamientos en las ecuaciones de Cauchy-Navier, para el caso axisimétrico, asumiendo que las fuerzas másicas son nulas: $b_r = b_z = 0$:
cn1 = (lamda + G)*diff(e,r) + G*(lapl(ur) - ur/r**2)
cn2 = (lamda + G)*diff(e,z) + G*lapl(w)

para verificar que estas ecuaciones se convierten respectivamente en:
\begin{align}
    0 &= 0 \\
    (1 - \nu)\nabla^4\phi(r,z) &= 0
\end{align}

In [15]:
imprimir("", factor(cn1 - 0), " = 0")

<IPython.core.display.Math object>

In [16]:
imprimir("", factor(cn2 - (1-nu)*bphi), " = 0")

<IPython.core.display.Math object>